# Análisis de Componentes Principales (PCA) para Decisiones Inmobiliarias

Este notebook implementa un análisis de componentes principales (PCA) para reducir la dimensionalidad de variables que influyen en decisiones de compra vs. alquiler de inmuebles.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configuración para visualizaciones
plt.style.use('seaborn')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 8)

## Carga y Exploración de Datos

In [ ]:
# Cargar los datos
df = pd.read_csv('../data/comprar_alquilar.csv')

# Exploración inicial
print("Información del dataset:")
df.info()

print("\nEstadísticas descriptivas:")
df.describe()

In [ ]:
# Visualizar la distribución de la variable objetivo
plt.figure(figsize=(8, 6))
sns.countplot(x='comprar', data=df)
plt.title('Distribución de Decisiones: Comprar vs Alquilar')
plt.xlabel('Decisión (0=Alquilar, 1=Comprar)')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
# Matriz de correlación
plt.figure(figsize=(14, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación')
plt.savefig('../results/comprar_alquilar_correlation_matrix.png')
plt.show()

## Preparación de Datos y Aplicación de PCA

In [ ]:
# Separar características y variable objetivo
X = df.drop('comprar', axis=1)
y = df['comprar']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Guardar el scaler para uso futuro
import joblib
joblib.dump(scaler, '../models/scaler_comprar_alquilar.pkl')

In [ ]:
# Aplicar PCA
pca = PCA()
pca.fit(X_scaled)

# Visualizar la varianza explicada acumulada
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, marker='o', linestyle='-')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% de varianza explicada')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('Varianza Explicada Acumulada vs. Número de Componentes')
plt.grid(True)
plt.legend()
plt.savefig('../results/comprar_alquilar_varianza_acumulada.png')
plt.show()

# Determinar el número de componentes para explicar el 95% de la varianza
n_components = np.argmax(cumulative_variance_ratio >= 0.95) + 1
print(f"\nNúmero de componentes para explicar el 95% de la varianza: {n_components}")

In [ ]:
# Aplicar PCA con el número óptimo de componentes
pca_optimal = PCA(n_components=n_components)
X_pca = pca_optimal.fit_transform(X_scaled)

# Guardar el modelo PCA
joblib.dump(pca_optimal, '../models/modelo_pca_comprar_alquilar.pkl')

## Visualización de Resultados del PCA

In [ ]:
# Visualizar los datos en las dos primeras componentes principales
plt.figure(figsize=(10, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y, palette='viridis', s=100, alpha=0.7)
plt.title('Visualización de Datos en las Dos Primeras Componentes Principales')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Comprar', labels=['Alquilar', 'Comprar'])
plt.grid(True)
plt.savefig('../results/comprar_alquilar_pca_visualization.png')
plt.show()

In [ ]:
# Analizar la contribución de cada característica a las componentes principales
components = pd.DataFrame(
    pca_optimal.components_,
    columns=X.columns
)

plt.figure(figsize=(14, 10))
sns.heatmap(components, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Contribución de Características a las Componentes Principales')
plt.xlabel('Características')
plt.ylabel('Componentes Principales')
plt.savefig('../results/comprar_alquilar_pca_components.png')
plt.show()

## Evaluación del Modelo con PCA

In [ ]:
# Evaluar el rendimiento de un modelo con las componentes principales
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

# Entrenar un modelo de clasificación
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrecisión del modelo con PCA: {accuracy:.4f}")
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))

# Guardar el modelo Random Forest con PCA
joblib.dump(rf, '../models/modelo_rf_pca_comprar_alquilar.pkl')

In [ ]:
# Comparar con el modelo sin reducción de dimensionalidad
X_original_train, X_original_test, y_original_train, y_original_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

rf_original = RandomForestClassifier(random_state=42)
rf_original.fit(X_original_train, y_original_train)

y_original_pred = rf_original.predict(X_original_test)
accuracy_original = accuracy_score(y_original_test, y_original_pred)

print(f"\nPrecisión del modelo sin PCA: {accuracy_original:.4f}")
print("\nInforme de clasificación sin PCA:")
print(classification_report(y_original_test, y_original_pred))

print(f"\nDiferencia de precisión (PCA - Original): {accuracy - accuracy_original:.4f}")

## Conclusiones

- El análisis de componentes principales (PCA) permitió reducir la dimensionalidad de los datos manteniendo el 95% de la varianza con solo [n] componentes.
- La visualización de los datos en las dos primeras componentes principales muestra una clara separación entre las decisiones de compra y alquiler.
- El modelo de Random Forest entrenado con las componentes principales logró una precisión comparable al modelo entrenado con todas las características originales, lo que demuestra la efectividad de la reducción de dimensionalidad.
- Las características que más contribuyen a las primeras componentes principales son [completar con los resultados del análisis].
- La reducción de dimensionalidad mediante PCA facilita la interpretación de los factores que influyen en las decisiones inmobiliarias y permite construir modelos más eficientes.